# General

## CRISP-DM

## ETL

### (E)xtraction


#### Web of Science/Zoological Records


#### TreatmentBank (Plazi)


#### Zoobank


#### Exploratory Analyses: Extraction


### (T)ransform


#### Web of Science/Zoological Records


#### TreatmentBank (Plazi)


#### Zoobank


#### SHERPA/RoMEO
At this point, I'm not sure if I'll include it.


#### Exploratory Analyses: Transform


### (L)oad


## Analysis


### Exploratory Analyses: Final


### Question #1


### Question #2


### Question #3

### Question #4

### Question #5

## Conclusion

## References
I'm not sure if i'll maintain this section or only add these to the README.md